In [2]:
import xarray as xr

In [6]:
spear_daily = xr.open_dataset('/data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/t_ref_min/output/monthly_atmos_daily.t_ref_min.1991.01.nc')
print(spear_daily)

<xarray.Dataset>
Dimensions:     (time: 12, ens: 15, bnds: 2, lat: 70, lon: 112)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 20.25 20.75 21.25 21.75 ... 53.75 54.25 54.75
  * lon         (lon) float64 230.3 230.9 231.6 232.2 ... 298.4 299.1 299.7
  * time        (time) object 1991-01-31 00:00:00 ... 1991-12-31 00:00:00
Dimensions without coordinates: ens
Data variables:
    average_DT  (time, ens) timedelta64[ns] ...
    lat_bnds    (time, lat, bnds) float64 ...
    lon_bnds    (time, lon, bnds) float64 ...
    t_ref_min   (time, ens, lat, lon) float32 ...
    time_bnds   (time, bnds) timedelta64[ns] ...
Attributes:
    filename:      atmos_daily.19910101-19911231.t_ref_min.nc
    title:         i19910101_OTA_IceAtmRes_L33_ens_01_05
    grid_type:     regular
    grid_tile:     N/A
    history:       Tue Oct 10 17:06:46 2023: ncecat -d lon,230.0,300.0 -d lat...
    code_version:  $Name: bronx-10_performance_z1l $
    NCO:           netCDF Operators

In [1]:
import xarray as xr

# Load the dataset
file_path = "/data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.01.nc"
dataset = xr.open_dataset(file_path)

# Display all aspects of the data
print("Dataset Dimensions:")
print(dataset.dims)
print("\nDataset Coordinates:")
print(dataset.coords)
print("\nDataset Variables:")
print(dataset.data_vars)

# Calculate and print the range of latitude and longitude
lat_range = (dataset.lat.min().item(), dataset.lat.max().item())
lon_range = (dataset.lon.min().item(), dataset.lon.max().item())
print("\nLatitude Range:", lat_range)
print("Longitude Range:", lon_range)

# Print attributes
print("\nDataset Attributes:")
print(dataset.attrs)


Dataset Dimensions:
Frozen({'ens': 15, 'time': 365, 'bnds': 2, 'lat': 70, 'lon': 112})

Dataset Coordinates:
Coordinates:
  * bnds     (bnds) float64 1.0 2.0
  * lat      (lat) float64 20.25 20.75 21.25 21.75 ... 53.25 53.75 54.25 54.75
  * lon      (lon) float64 230.3 230.9 231.6 232.2 ... 297.8 298.4 299.1 299.7
  * time     (time) object 1991-01-01 12:00:00 ... 1991-12-31 12:00:00

Dataset Variables:
Data variables:
    average_DT  (ens, time) timedelta64[ns] ...
    average_T1  (ens, time) datetime64[ns] ...
    average_T2  (ens, time) datetime64[ns] ...
    lat_bnds    (lat, bnds) float64 ...
    lon_bnds    (lon, bnds) float64 ...
    precip      (ens, time, lat, lon) float32 ...
    time_bnds   (time, bnds) timedelta64[ns] ...

Latitude Range: (20.25, 54.75000000000001)
Longitude Range: (230.3125, 299.6875)

Dataset Attributes:
{'filename': 'atmos_daily.19910101-19911231.precip.nc', 'title': 'i19910101_OTA_IceAtmRes_L33_ens_01_05', 'grid_type': 'regular', 'grid_tile': 'N/A', 'hi

In [22]:
import xarray as xr
import pandas as pd
import os
from datetime import datetime

# Define the locations and their coordinates
locations = {
    "Stillhouse-Hollow": (-97.584652, 31.023373),
    "Granger": (-97.349046, 30.699781),
    "Georgetown": (-97.75206, 30.674299),
    "Belton": (-97.504701, 31.138321),
}

# Path to the forecast data
forecast_file = "/data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.01.nc"

# Output directory
output_dir = "/data/aydin/project/transition/USA/time_series/data/location_output"
os.makedirs(output_dir, exist_ok=True)

# Load the NetCDF file
ds = xr.open_dataset(forecast_file)

# Convert longitudes to the 0-360 range if necessary
ds["lon"] = ds["lon"].where(ds["lon"] >= 0, ds["lon"] + 360)

# Convert cftime to datetime64
if isinstance(ds["time"].values[0], cftime.datetime):
    ds["time"] = ds["time"].to_index().to_datetimeindex()

# Loop through each location
for location, (lon, lat) in locations.items():
    print(f"Processing location: {location} (Lon: {lon}, Lat: {lat})")
    
    # Find the nearest grid point
    nearest_lon = ds.sel(lon=lon, method="nearest")["lon"].values
    nearest_lat = ds.sel(lat=lat, method="nearest")["lat"].values
    
    # Extract the data for the nearest grid point
    point_data = ds.sel(lon=nearest_lon, lat=nearest_lat)
    
    # Convert precipitation from kg/m²/s to mm/day
    point_data["precip"] *= 86400  # Apply conversion factor
    
    # Calculate the ensemble mean
    ensemble_mean = point_data["precip"].mean(dim="ens")
    
    # Create a DataFrame for ensemble members
    df_ens = point_data.to_dataframe().reset_index()
    df_ens["ensemble"] = df_ens["ens"]  # Add an ensemble member column
    
    # Create a DataFrame for the ensemble mean
    df_mean = ensemble_mean.to_dataframe(name="precip").reset_index()
    df_mean["ens"] = "mean"
    df_mean["ensemble"] = "mean"
    
    # Combine ensemble member and mean data
    df = pd.concat([df_ens, df_mean], ignore_index=True)
    
    # Ensure 'time' is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df["time"]):
        df["time"] = [datetime(t.year, t.month, t.day) for t in df["time"]]
    
    # Add initial date
    df["initial_date"] = df["time"].min()
    
    # Calculate lead time in days
    df["lead_time"] = (df["time"] - df["initial_date"]).dt.days
    
    # Rearrange columns
    df = df[["initial_date", "lead_time", "time", "ensemble", "precip"]]
    
    # Construct the output file path
    output_file = os.path.join(output_dir, f"{location}_forecast_timeseries.csv")
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    print(f"Saved: {output_file}")

print("Processing completed for all locations.")


/tmp/ipykernel_706256/2208642764.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()


Processing location: Stillhouse-Hollow (Lon: -97.584652, Lat: 31.023373)
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Stillhouse-Hollow_forecast_timeseries.csv
Processing location: Granger (Lon: -97.349046, Lat: 30.699781)
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Granger_forecast_timeseries.csv
Processing location: Georgetown (Lon: -97.75206, Lat: 30.674299)
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Georgetown_forecast_timeseries.csv
Processing location: Belton (Lon: -97.504701, Lat: 31.138321)
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Belton_forecast_timeseries.csv
Processing completed for all locations.


In [17]:
import os

# Define the folder path
folder_path = "/data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip"

# List all .nc files directly in the folder
nc_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".nc")]

# Join all file paths into a single string separated by spaces (for easy copying)
all_files = " ".join(nc_files)

# Print the result
print(all_files)


/data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.01.nc /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.02.nc /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.03.nc /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.04.nc /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.05.nc /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.06.nc /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.07.nc /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.08.nc /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR

In [21]:
import xarray as xr
import pandas as pd
import os
from datetime import datetime
import cftime  # Import cftime directly for handling cftime.datetime

# Define the locations and their coordinates
locations = {
    "Stillhouse-Hollow": (-97.584652, 31.023373),
    "Granger": (-97.349046, 30.699781),
    "Georgetown": (-97.75206, 30.674299),
    "Belton": (-97.504701, 31.138321),
}

# Path to the forecast data folder
forecast_folder = "/data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip"

# Output directory
output_parent_dir = "/data/aydin/project/transition/USA/time_series/data/location_output"
os.makedirs(output_parent_dir, exist_ok=True)

# List all .nc files directly inside the folder (no subfolders)
nc_files = [os.path.join(forecast_folder, f) for f in os.listdir(forecast_folder) if f.endswith(".nc")]

# Process each location
for location, (lon, lat) in locations.items():
    # Create a folder for the location
    location_dir = os.path.join(output_parent_dir, location)
    os.makedirs(location_dir, exist_ok=True)

    # Process each NetCDF file
    for forecast_file in nc_files:
        print(f"Processing file: {forecast_file} for location: {location}")

        # Load the NetCDF file
        ds = xr.open_dataset(forecast_file)

        # Convert longitudes to the 0-360 range if necessary
        ds["lon"] = ds["lon"].where(ds["lon"] >= 0, ds["lon"] + 360)

        # Convert cftime to datetime64
        if isinstance(ds["time"].values[0], cftime.datetime):
            ds["time"] = ds["time"].to_index().to_datetimeindex()

        # Find the nearest grid point
        nearest_lon = ds.sel(lon=lon, method="nearest")["lon"].values
        nearest_lat = ds.sel(lat=lat, method="nearest")["lat"].values

        # Extract the data for the nearest grid point
        point_data = ds.sel(lon=nearest_lon, lat=nearest_lat)

        # Convert precipitation from kg/m²/s to mm/day
        point_data["precip"] *= 86400  # Apply conversion factor

        # Calculate the ensemble mean
        ensemble_mean = point_data["precip"].mean(dim="ens")

        # Create a DataFrame for ensemble members
        df_ens = point_data.to_dataframe().reset_index()
        df_ens["ensemble"] = df_ens["ens"]  # Add an ensemble member column

        # Create a DataFrame for the ensemble mean
        df_mean = ensemble_mean.to_dataframe(name="precip").reset_index()
        df_mean["ens"] = "mean"
        df_mean["ensemble"] = "mean"

        # Combine ensemble member and mean data
        df = pd.concat([df_ens, df_mean], ignore_index=True)

        # Ensure 'time' is in datetime format
        if not pd.api.types.is_datetime64_any_dtype(df["time"]):
            df["time"] = [datetime(t.year, t.month, t.day) for t in df["time"]]

        # Add initial date
        df["initial_date"] = df["time"].min()

        # Calculate lead time in days
        df["lead_time"] = (df["time"] - df["initial_date"]).dt.days

        # Rearrange columns
        df = df[["initial_date", "lead_time", "time", "ensemble", "precip"]]

        # Construct the output file path
        output_file = os.path.join(location_dir, f"{os.path.basename(forecast_file).replace('.nc', '_timeseries.csv')}")

        # Save to CSV
        df.to_csv(output_file, index=False)
        print(f"Saved: {output_file}")

print("Processing completed for all locations and files.")


Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.01.nc for location: Stillhouse-Hollow
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Stillhouse-Hollow/atmos_daily.precip.1991.01_timeseries.csv
Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.02.nc for location: Stillhouse-Hollow


/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()
/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()


Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Stillhouse-Hollow/atmos_daily.precip.1991.02_timeseries.csv
Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.03.nc for location: Stillhouse-Hollow


/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()


Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Stillhouse-Hollow/atmos_daily.precip.1991.03_timeseries.csv
Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.1991.04.nc for location: Stillhouse-Hollow


...

/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()
/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()
/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time betwee

Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Belton/atmos_daily.precip.2022.04_timeseries.csv
Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.2022.05.nc for location: Belton
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Belton/atmos_daily.precip.2022.05_timeseries.csv
Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.2022.06.nc for location: Belton
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Belton/atmos_daily.precip.2022.06_timeseries.csv
Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.2022.07.nc for location: Belton
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Belton/atmos_daily.precip.2022.07_timeseries.csv
Processing file: /data/aydin/project/Ensem

/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()
/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()
/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time betwee

Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Belton/atmos_daily.precip.2022.11_timeseries.csv
Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/precip/atmos_daily.precip.2022.12.nc for location: Belton
Saved: /data/aydin/project/transition/USA/time_series/data/location_output/Belton/atmos_daily.precip.2022.12_timeseries.csv
Processing completed for all locations and files.


/tmp/ipykernel_706256/4135458709.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()


In [5]:
import xarray as xr
import pandas as pd
import os
from datetime import datetime
import cftime  # Import cftime directly for handling cftime.datetime

# Define the locations and their coordinates
locations = {
    "Stillhouse-Hollow": (-97.584652, 31.023373),
    "Granger": (-97.349046, 30.699781),
    "Georgetown": (-97.75206, 30.674299),
    "Belton": (-97.504701, 31.138321),
}

# Path to the forecast data folder
forecast_folder = "/data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/t_ref_min/output"

# Output directory
output_parent_dir = "/data/aydin/project/transition/USA/time_series/data/location_output_tmin"
os.makedirs(output_parent_dir, exist_ok=True)

# List all .nc files directly inside the folder (no subfolders)
nc_files = [os.path.join(forecast_folder, f) for f in os.listdir(forecast_folder) if f.endswith(".nc")]

# Process each location
for location, (lon, lat) in locations.items():
    # Create a folder for the location
    location_dir = os.path.join(output_parent_dir, location)
    os.makedirs(location_dir, exist_ok=True)

    # Process each NetCDF file
    for forecast_file in nc_files:
        print(f"Processing file: {forecast_file} for location: {location}")

        # Load the NetCDF file
        ds = xr.open_dataset(forecast_file)

        # Convert longitudes to the 0-360 range if necessary
        ds["lon"] = ds["lon"].where(ds["lon"] >= 0, ds["lon"] + 360)

        # Convert cftime to datetime64
        if isinstance(ds["time"].values[0], cftime.datetime):
            ds["time"] = ds["time"].to_index().to_datetimeindex()

        # Find the nearest grid point
        nearest_lon = ds.sel(lon=lon, method="nearest")["lon"].values
        nearest_lat = ds.sel(lat=lat, method="nearest")["lat"].values

        # Extract the data for the nearest grid point
        point_data = ds.sel(lon=nearest_lon, lat=nearest_lat)

        # Extract temperature data
        temperature_data = point_data["t_ref_max"]

        # Calculate the ensemble mean
        ensemble_mean = temperature_data.mean(dim="ens")

        # Create a DataFrame for ensemble members
        df_ens = temperature_data.to_dataframe().reset_index()
        df_ens["ensemble"] = df_ens["ens"]  # Add an ensemble member column

        # Create a DataFrame for the ensemble mean
        df_mean = ensemble_mean.to_dataframe(name="t_ref_max").reset_index()
        df_mean["ens"] = "mean"
        df_mean["ensemble"] = "mean"

        # Combine ensemble member and mean data
        df = pd.concat([df_ens, df_mean], ignore_index=True)

        # Ensure 'time' is in datetime format
        if not pd.api.types.is_datetime64_any_dtype(df["time"]):
            df["time"] = [datetime(t.year, t.month, t.day) for t in df["time"]]

        # Add initial date
        df["initial_date"] = df["time"].min()

        # Calculate lead time in days
        df["lead_time"] = (df["time"] - df["initial_date"]).dt.days

        # Rearrange columns
        df = df[["initial_date", "lead_time", "time", "ensemble", "t_ref_max"]]

        # Construct the output file path
        output_file = os.path.join(location_dir, f"{os.path.basename(forecast_file).replace('.nc', '_timeseries.csv')}")

        # Save to CSV
        df.to_csv(output_file, index=False)
        print(f"Saved: {output_file}")

print("Processing completed for all locations and files.")


Processing file: /data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/t_ref_min/output/monthly_atmos_daily.t_ref_min.1991.01.nc for location: Stillhouse-Hollow


/tmp/ipykernel_1294190/3024081975.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds["time"] = ds["time"].to_index().to_datetimeindex()


KeyError: 't_ref_max'

In [ ]:
import xarray as xr
import pandas as pd
import os
from datetime import datetime
import cftime  # Import cftime directly for handling cftime.datetime

# Define the locations and their coordinates
locations = {
    "Stillhouse-Hollow": (-97.584652, 31.023373),
    "Granger": (-97.349046, 30.699781),
    "Georgetown": (-97.75206, 30.674299),
    "Belton": (-97.504701, 31.138321),
}

# Path to the forecast data folder
forecast_folder = "/data/aydin/project/EnsembleForecastVerification/Data/RawForecasts/SPEAR/Daily/t_ref_min/output"

# Output directory
output_parent_dir = "/data/aydin/project/transition/USA/time_series/data/location_output_tmin"
os.makedirs(output_parent_dir, exist_ok=True)

# List all .nc files directly inside the folder (no subfolders)
nc_files = [os.path.join(forecast_folder, f) for f in os.listdir(forecast_folder) if f.endswith(".nc")]

# Process each location
for location, (lon, lat) in locations.items():
    # Create a folder for the location
    location_dir = os.path.join(output_parent_dir, location)
    os.makedirs(location_dir, exist_ok=True)

    # Process each NetCDF file
    for forecast_file in nc_files:
        print(f"Processing file: {forecast_file} for location: {location}")

        # Load the NetCDF file
        ds = xr.open_dataset(forecast_file)

        # Convert longitudes to the 0-360 range if necessary
        ds["lon"] = ds["lon"].where(ds["lon"] >= 0, ds["lon"] + 360)

        # Convert cftime to datetime64
        if isinstance(ds["time"].values[0], cftime.datetime):
            ds["time"] = ds["time"].to_index().to_datetimeindex()

        # Find the nearest grid point
        nearest_lon = ds.sel(lon=lon, method="nearest")["lon"].values
        nearest_lat = ds.sel(lat=lat, method="nearest")["lat"].values

        # Extract the data for the nearest grid point
        point_data = ds.sel(lon=nearest_lon, lat=nearest_lat)

        # Extract minimum temperature data
        tmin_data = point_data["t_ref_min"]

        # Calculate the ensemble mean
        ensemble_mean = tmin_data.mean(dim="ens")

        # Create a DataFrame for ensemble members
        df_ens = tmin_data.to_dataframe().reset_index()
        df_ens["ensemble"] = df_ens["ens"]  # Add an ensemble member column

        # Create a DataFrame for the ensemble mean
        df_mean = ensemble_mean.to_dataframe(name="t_ref_min").reset_index()
        df_mean["ens"] = "mean"
        df_mean["ensemble"] = "mean"

        # Combine ensemble member and mean data
        df = pd.concat([df_ens, df_mean], ignore_index=True)

        # Ensure 'time' is in datetime format
        if not pd.api.types.is_datetime64_any_dtype(df["time"]):
            df["time"] = [datetime(t.year, t.month, t.day) for t in df["time"]]

        # Add initial date
        df["initial_date"] = df["time"].min()

        # Calculate lead time in days
        df["lead_time"] = (df["time"] - df["initial_date"]).dt.days

        # Rearrange columns
        df = df[["initial_date", "lead_time", "time", "ensemble", "t_ref_min"]]

        # Construct the output file path
        output_file = os.path.join(location_dir, f"{os.path.basename(forecast_file).replace('.nc', '_timeseries.csv')}")

        # Save to CSV
        df.to_csv(output_file, index=False)
        print(f"Saved: {output_file}")

print("Processing completed for all locations and files.")
